# Setup

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

# Data Loading

In [3]:
train = pd.read_csv('/content/drive/MyDrive/ME /SinoPac/30_Training Dataset_V2/training_data.csv') # training data
testpub = pd.read_csv('/content/drive/MyDrive/ME /SinoPac/public_dataset.csv')                 # testing data (public leader board)
subpub = pd.read_csv('/content/drive/MyDrive/ME /SinoPac/public_submission_template.csv')      # submission file

In [4]:
train.head()

,ID,縣市,鄉鎮市區,路名,土地面積,使用分區,移轉層次,總樓層數,主要用途,主要建材,...,建物面積,車位面積,車位個數,橫坐標,縱坐標,備註,主建物面積,陽台面積,附屬建物面積,單價
0,TR-1,台北市,大安區,敦化南路二段,-0.256716,None,11,11,住家用,鋼筋混凝土造,...,-0.174154,-0.819326,0.0,305266,2768378,NaN,0.393926,0.183700,-0.438452,4.627714
1,TR-2,台北市,萬華區,水源路,0.100134,None,7,12,住家用,鋼筋混凝土造,...,0.314204,-0.819326,0.0,300677,2767990,NaN,-0.316131,0.608577,-0.438452,1.887258
2,TR-3,高雄市,鳳山區,北忠街,0.181921,None,10,15,集合住宅,其他,...,0.423366,0.161624,1.0,184815,2504666,NaN,-0.098871,-0.360620,1.525881,1.489072
3,TR-4,新北市,新莊區,福前街,0.085594,None,9,14,集合住宅,鋼筋混凝土造,...,0.164249,0.524653,1.0,296653,2772355,NaN,-0.071147,0.315088,0.231984,2.051217
4,TR-5,新北市,板橋區,文化路一段,-0.938116,None,41,43,住家用,鋼骨造,...,0.985839,0.532377,1.0,297377,2768472,NaN,0.791954,1.719400,-0.438452,3.269198


# Data Preprocessing

In [5]:
# Check missing value
print(train.isnull().sum())
print(testpub.isnull().sum())

ID            0
縣市            0
鄉鎮市區          0
路名            0
土地面積          0
使用分區          0
移轉層次          0
總樓層數          0
主要用途          0
主要建材          0
建物型態          0
屋齡            0
建物面積          0
車位面積          0
車位個數          0
橫坐標           0
縱坐標           0
備註        11659
主建物面積         0
陽台面積          0
附屬建物面積        0
單價            0
dtype: int64
ID           0
縣市           0
鄉鎮市區         0
路名           0
土地面積         0
使用分區         0
移轉層次         0
總樓層數         0
主要用途         0
主要建材         0
建物型態         0
屋齡           0
建物面積         0
車位面積         0
車位個數         0
橫坐標          0
縱坐標          0
備註        5828
主建物面積        0
陽台面積         0
附屬建物面積       0
dtype: int64


In [6]:
train = train.drop(['備註'], axis=1)
testpub = testpub.drop(['備註'], axis=1)
print(train.isnull().sum())
print(testpub.isnull().sum())

ID        0
縣市        0
鄉鎮市區      0
路名        0
土地面積      0
使用分區      0
移轉層次      0
總樓層數      0
主要用途      0
主要建材      0
建物型態      0
屋齡        0
建物面積      0
車位面積      0
車位個數      0
橫坐標       0
縱坐標       0
主建物面積     0
陽台面積      0
附屬建物面積    0
單價        0
dtype: int64
ID        0
縣市        0
鄉鎮市區      0
路名        0
土地面積      0
使用分區      0
移轉層次      0
總樓層數      0
主要用途      0
主要建材      0
建物型態      0
屋齡        0
建物面積      0
車位面積      0
車位個數      0
橫坐標       0
縱坐標       0
主建物面積     0
陽台面積      0
附屬建物面積    0
dtype: int64


In [7]:
# Check column Dtype
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11751 entries, 0 to 11750
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      11751 non-null  object 
 1   縣市      11751 non-null  object 
 2   鄉鎮市區    11751 non-null  object 
 3   路名      11751 non-null  object 
 4   土地面積    11751 non-null  float64
 5   使用分區    11751 non-null  object 
 6   移轉層次    11751 non-null  int64  
 7   總樓層數    11751 non-null  int64  
 8   主要用途    11751 non-null  object 
 9   主要建材    11751 non-null  object 
 10  建物型態    11751 non-null  object 
 11  屋齡      11751 non-null  float64
 12  建物面積    11751 non-null  float64
 13  車位面積    11751 non-null  float64
 14  車位個數    11751 non-null  float64
 15  橫坐標     11751 non-null  int64  
 16  縱坐標     11751 non-null  int64  
 17  主建物面積   11751 non-null  float64
 18  陽台面積    11751 non-null  float64
 19  附屬建物面積  11751 non-null  float64
 20  單價      11751 non-null  float64
dtypes: float64(9), int64(4), object(8)


In [8]:
object_columns = train.select_dtypes(include=['object']).columns
for column in object_columns:
    print(f"{column}, Dtype: {train[column].dtype}")

ID, Dtype: object
縣市, Dtype: object
鄉鎮市區, Dtype: object
路名, Dtype: object
使用分區, Dtype: object
主要用途, Dtype: object
主要建材, Dtype: object
建物型態, Dtype: object


In [9]:
train = pd.DataFrame(train)
tetspub = pd.DataFrame(testpub)

train['ID'] = train['ID'].apply(lambda x: int(x.replace('TR-', '')) if isinstance(x, str) and x.startswith('TR-') else x)
testpub['ID'] = testpub['ID'].apply(lambda x: int(x.replace('PU-', '')) if isinstance(x, str) and x.startswith('PU-') else x)

In [10]:
label_encoder = LabelEncoder()
object_columns = train.select_dtypes(include=['object']).columns

for column in object_columns:
    train[column] = label_encoder.fit_transform(train[column])

for column in object_columns:
    testpub[column] = label_encoder.fit_transform(testpub[column])

In [11]:
# Data Spliting
X = train.drop('單價', axis=1)
y = train['單價']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Model Training

In [ ]:
# Linear Regression
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred)
print(mape)

0.31284566604493164


In [13]:
# XGBoost
model = XGBRegressor()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mape = mean_absolute_percentage_error(y_test, y_pred)
print(mape)

0.10348032972478752


# Test

In [14]:
subpub = pd.DataFrame(subpub)
pred = model.predict(testpub)
subpub['predicted_price'] = pred

In [15]:
subpub.to_csv('/content/drive/MyDrive/ME /SinoPac/submission_2.csv', index=False)